load env

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

True

## Configuration

## State

In [2]:
import uuid
from typing import Any, Optional, Union, Literal
from langchain_core.documents import Document

def reduce_docs(
    existing: Optional[list[Document]],
    new: Union[
        list[Document],
        list[dict[str, Any]],
        list[str],
        str,
        Literal["delete"],
    ],
) -> list[Document]:
    """Reduce and process documents based on the input type.

    This function handles various input types and converts them into a sequence of Document objects.
    It also combines existing documents with the new one based on the document ID.

    Args:
        existing (Optional[Sequence[Document]]): The existing docs in the state, if any.
        new (Union[Sequence[Document], Sequence[dict[str, Any]], Sequence[str], str, Literal["delete"]]):
            The new input to process. Can be a sequence of Documents, dictionaries, strings, or a single string.
    """
    if new == "delete":
        return []

    existing_list = list(existing) if existing else []
    if isinstance(new, str):
        return existing_list + [
            Document(page_content=new, metadata={"uuid": str(uuid.uuid4())})
        ]

    new_list = []
    if isinstance(new, list):
        existing_ids = set(doc.metadata.get("uuid") for doc in existing_list)
        for item in new:
            if isinstance(item, str):
                item_id = str(uuid.uuid4())
                new_list.append(Document(page_content=item, metadata={"uuid": item_id}))
                existing_ids.add(item_id)

            elif isinstance(item, dict):
                metadata = item.get("metadata", {})
                item_id = metadata.get("uuid", str(uuid.uuid4()))

                if item_id not in existing_ids:
                    new_list.append(
                        Document(**item, metadata={**metadata, "uuid": item_id})
                    )
                    existing_ids.add(item_id)

            elif isinstance(item, Document):
                item_id = item.metadata.get("uuid")
                if item_id is None:
                    item_id = str(uuid.uuid4())
                    new_item = item.model_copy(update={"metadata": {"uuid": item_id}})
                else:
                    new_item = item

                if item_id not in existing_ids:
                    new_list.append(new_item)
                    existing_ids.add(item_id)


In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, Annotated
from langchain_core.messages import AnyMessage
from langchain_core.documents import Document
from langgraph.graph import add_messages

class Router(BaseModel):
    logic: str = Field(default="", description="The logic of the router.")
    type: Annotated[Literal["more-info", "langchain", "general"]] = Field(default="general", description="The type of the router.")
    

class InputState(BaseModel):
    messages: Annotated[list[AnyMessage], add_messages] = Field(default_factory=list, description="Accumulated messages with unique IDs.")

class AgentState(InputState):
    router: Router = Field(default=Router(type="general", logic=""), description="Router classification of the user's query.")
    steps: list[str] = Field(default_factory=list, description="A list of steps in the research plan.")
    documents: Annotated[list[Document], reduce_docs] = Field(default_factory=list, description="Populated by the retriever. This is a list of documents that the agent can reference.")
    answer: str = Field(default="", description="Final answer. Useful for evaluations")
    query: str = Field(default="", description="The query of the user.")

## Tools

### Router

## Graph